In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园：使用Bytetrack进行视频目标跟踪
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_video_object_tracking_serve.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_video_object_tracking_serve.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_video_object_tracking_serve.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI Workbench中打开
    </a>
    (建议使用Python-3 CPU笔记本)
  </td>
</table>

注意：此笔记本已在以下环境中进行测试：

Python版本= 3.9

## 概述

该 Jupyter 笔记本提供了部署视频目标跟踪模型到 Vertex AI Endpoint 资源的逐步操作指南，使用开源的 [ByteTrack](https://github.com/ifzhang/ByteTrack) 目标跟踪算法。

### 目标
* 设置 Vertex AI Endpoint 资源，并选择以下其中一个子选项：
    * TensorFlow Vision [笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_tfvision_image_object_detection.ipynb) 或
    * Google 专有 [笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_proprietary_image_object_detection.ipynb)
<br></br>
* 测试集成跟踪模型
    * 上传模型到模型注册表
    * 部署已上传的模型
    * 运行批量预测
    * 验证和可视化跟踪结果
<br></br>
* 清理资源

### 成本

本教程使用 Google Cloud 中的可计费组件：

* Vertex AI
* Cloud Storage

了解有关 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定价](https://cloud.google.com/storage/pricing) 的信息，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

## 安装

安装以下所需的软件包以执行这个笔记本。

In [ ]:
! pip install --upgrade pip
! pip install fastapi==0.96.0
! pip install google-cloud-aiplatform==1.25.0
! pip install google-cloud-storage==2.9.0
! pip install tensorflow==2.11.0
! pip install uvicorn==0.22.0

仅限Colab
如果您正在使用Workbench，请运行以下命令并跳过本部分。

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

如果您在本地运行此笔记本，您将需要安装[Cloud SDK](https://cloud.google.com/sdk)和[gsutil](https://cloud.google.com/storage/docs/gsutil_install)。

## 开始之前

### 设置您的Google Cloud项目

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得$300的免费信用，可用于您的计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预计使用量生成费用估算。

3. [启用Artifact Registry](https://cloud.google.com/artifact-registry/docs/enable-service)并[创建存储Docker镜像的仓库](https://cloud.google.com/artifact-registry/docs/repositories/create-repos)。

4. [创建一个GCS存储桶](https://cloud.google.com/storage/docs/creating-buckets)用于存储实验输出。

5. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

6. [创建一个服务账号](https://cloud.google.com/iam/docs/service-accounts-create?&_ga=2.233472348.-356102079.1688744268#iam-service-accounts-create-console)，为其分配Vertex AI User和Storage Object Admin角色，用于部署微调模型到Vertex AI终端。[查看如何向您的服务账号授予Cloud Storage权限](https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples)。

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

要更新您的模型工件而无需重新构建容器，您必须将模型工件和任何自定义代码上传到云存储中。

请在下面设置您的云存储桶的名称。它必须在所有云存储桶中保持唯一。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

只有在您的存储桶尚不存在时才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

设置剩余的变量

In [ ]:
# Cloud project setup.

# The folder in the GCS bucket with input videos.
# Fill it without the 'gs://' prefix.
INPUT_GCS_FOLDER = ""  # @param {type:"string"}

# The video filename for the videos to be process.
# Fill it without the 'gs://' prefix.
VIDEO_FILE_NAME = ""  # @param {type:"string"}

# The video filename extension like .mp4. Please include period.
# Fill it without the 'gs://' prefix.
VIDEO_FILE_EXTENSION = ""  # @param {type:"string"}

# The folder in the GCS bucket where to store output videos and text
# annotations. Fill it without the 'gs://' prefix.
OUTPUT_GCS_FOLDER = ""  # @param {type:"string"}

# The Vertex IOD endpoint for object detection.
# It is like projects/<project_number>/locations/<location>/endpoints/<endpoint_id>"
DETECTION_ENDPOINT = ""  # @param {type:"string"}

# The label map for the Vertex IOD endpoint.
# It is the path to a .yaml in GCS.
# It is like gs://{BUCKET_NAME}/{FOLDER_NAME}/label_map.yaml
ENDPOINT_LABEL_MAP = ""  # @param {type:"string"}

# You can choose a region from https://cloud.google.com/about/locations.
# Only regions prefixed by "us", "asia", or "europe" are supported.
REGION = "us-central1"  # @param {type:"string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

# The pre-built docker images
SERVE_DOCKER_URI = (
    "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/vot-serve:latest"
)

# Prediction constants
PREDICTION_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
PREDICTION_MACHINE_TYPE = "n1-standard-4"

# The serving port.
SERVE_PORT = 7080

# The serving route.
SERVE_ROUTE = "/predictions/vot_serving"

# The service account you created in step-6 above.
# It is like "<account_name>@<project>.iam.gserviceaccount.com"
SERVICE_ACCOUNT = ""  # @param {type:"string"}

### 初始化Vertex AI SDK

为您的项目初始化Python的Vertex AI SDK。

In [ ]:
from google.cloud import aiplatform

# Init common setup.
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### 定义效用函数

In [ ]:
def get_job_name_with_datetime(prefix: str):
    """
    Generate a job name string with the current date and time appended.

    Args:
        prefix: The prefix string to use for the job name.

    Returns:
        str: The job name string in the format "{prefix}_{YYYYMMDD_HHMMSS}".
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model(
    project_id=None,
    detection_endpoint=None,
    label_map=None,
    output_bucket=None,
    model_type="CUSTOM",
    save_video_results=1,
):
    """
    Deploy a model to a real-time prediction endpoint.

    Args:
        detection_endpoint: The endpoint URL for object detection.
        label_map: Mapping of class IDs to class names.
        output_bucket: GCS bucket to save results.
        save_video_results: Whether to save video results.

    Returns:
        The created endpoint and deployed model objects.
    """
    task = "tracking"
    endpoint = aiplatform.Endpoint.create(display_name=f"{task}-endpoint")
    serving_env = {
        "MODEL_ID": "Bytetrack-Multi-Object-Tracking",
        "MODEL_TYPE": model_type,
        "PROJECT_ID": project_id,
        "DETECTION_ENDPOINT": detection_endpoint,
        "LABEL_MAP": label_map,
        "OUTPUT_BUCKET": output_bucket,
        "SAVE_VIDEO_RESULTS": save_video_results,
        "DEPLOY_SOURCE": "notebook",
    }
    model = aiplatform.Model.upload(
        display_name=task,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[SERVE_PORT],
        serving_container_predict_route=SERVE_ROUTE,
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=PREDICTION_MACHINE_TYPE,
        accelerator_type=PREDICTION_ACCELERATOR_TYPE,
        accelerator_count=1,
        service_account=SERVICE_ACCOUNT,
    )
    return endpoint, model

视频对象跟踪与本地端点

本节展示如何部署具有本地跟踪功能的链式IOD Vertex AI端点，并获取保存在文本文件中的预测结果。用户还可以选择将带注释的视频保存到他们的GCS存储桶中。

* 如果您尚未这样做，请使用以下方法设置一个Vertex AI端点资源：
    * TensorFlow Vision [笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_tfvision_image_object_detection.ipynb) or
    * Google 专有 [笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_proprietary_image_object_detection.ipynb)

In [ ]:
# The Vertex IOD endpoint for object detection.
# It is like projects/<project_number>/locations/<location>/endpoints/<endpoint_id>"
DETECTION_ENDPOINT = ""  # @param {type:"string"}

这是用于向在本地主机上运行的跟踪模型服务容器发出请求的本地URL。它指向SERVE_PORT端口上的/predictions路由，该路由将处理模型推断请求。

In [ ]:
LOCAL_SERVE_URL = f"http://localhost:{SERVE_PORT}/{SERVE_ROUTE}"

使用自定义模型IOD端点运行容器。

在单独的 shell 中运行服务容器。以下环境变量是必需的：

* `DETECTION_ENDPOINT`：Vertex AI IOD 检测端点地址。
* `LABEL_MAP`：用于训练 IOD 的标签映射 yaml 文件的 GCS URI。
* `OUTPUT_BUCKET`：用于存储实验输出的 GCS 路径。

In [ ]:
!nvidia-docker run -t --rm \
-p {SERVE_PORT}:{SERVE_PORT} \
-e DETECTION_ENDPOINT=f"{DETECTION_ENDPOINT}" \
-e LABEL_MAP=f"{ENDPOINT_LABEL_MAP}" \
-e OUTPUT_BUCKET=f"gs://{GCS_BUCKET}/{OUTPUT_GCS_FOLDER}" \
-e SAVE_VIDEO_RESULTS=1 \
-e CUDA_VISIBLE_DEVICES=0 \
{SERVE_DOCKER_URI}

### 使用AutoML训练的模型IOD端点来运行容器。

在单独的 shell 中运行提供容器。需要以下环境变量：

* `PROJECT_ID`: Google Cloud 项目 ID。
* `MODEL_TYPE`: CUSTOM 或 AUTOML。
* `DETECTION_ENDPOINT`: 来自 AUTOML 训练的 IOD 模型的 Vertex AI IOD 检测终端点地址。
* `OUTPUT_BUCKET`: 用于存储实验输出的 GCS 路径。

In [ ]:
!nvidia-docker run -t --rm \
-p {SERVE_PORT}:{SERVE_PORT} \
-e PROJECT_ID={PROJECT_ID} \
-e MODEL_TYPE={MODEL_TYPE}  \
-e DETECTION_ENDPOINT=f"{DETECTION_ENDPOINT}" \
-e OUTPUT_BUCKET=f"gs://{GCS_BUCKET}/{OUTPUT_GCS_FOLDER}" \
-e SAVE_VIDEO_RESULTS=1 \
-e CUDA_VISIBLE_DEVICES=0 \
{SERVE_DOCKER_URI}

### 本地测试端点并进行在线预测
本节展示如何向端点发出预测请求，以获取保存到文本文件和/或注释视频输出的检测和跟踪对象的轨迹ID和边界框坐标。

In [ ]:
import json

payload = json.dumps(
    {
        "instances": [
            {
                "video_uri": f"gs://{GCS_BUCKET}/{INPUT_GCS_FOLDER}/{VIDEO_FILE_NAME}1{VIDEO_FILE_EXTENSION}"
            },
        ]
    }
)
r = requests.post(
    LOCAL_SERVE_URL,
    data=payload,
    headers={"content-type": "application/json", "Accept-Charset": "UTF-8"},
)
preds = r.json()

print(preds)

部署Vertex AI端点和自定义IOD模型。

In [ ]:
endpoint, model = deploy_model(
    detection_endpoint="",  # @param {type:"string"}
    label_map="",  # @param {type:"string"}
    output_bucket="",  # @param {type:"string"}
    model_type="CUSTOM",  # @param {type:"string"}
    save_video_results=1,
)

### 部署 Vertex AI 端点和经 AutoML 训练的 IOD 模型。

In [ ]:
endpoint, model = deploy_model(
    project_id="",  # @param {type:"string"}
    detection_endpoint="",  # @param {type:"string"}
    model_type="AUTOML",  # @param {type:"string"}
    output_bucket="",  # @param {type:"string"}
)

### 使用远程端点进行在线预测。

In [ ]:
instances = [
    {
        "data": {
            "video_uri": f"gs://{GCS_BUCKET}/{INPUT_GCS_FOLDER}/{VIDEO_FILE_NAME}1{VIDEO_FILE_EXTENSION}"
        }
    },
]
preds = endpoint.predict(instances=instances).predictions
print(preds)

## 批量预测

### 设置批量预测的输入文件并上传到gs桶
提供jsonl格式的批量预测输入。

In [ ]:
INPUT_FILE = "instances.jsonl"
VIDEO_PATH_1 = (
    f"gs://{GCS_BUCKET}/{INPUT_GCS_FOLDER}/{VIDEO_FILE_NAME}1{VIDEO_FILE_EXTENSION}"
)
VIDEO_PATH_2 = (
    f"gs://{GCS_BUCKET}/{INPUT_GCS_FOLDER}/{VIDEO_FILE_NAME}2{VIDEO_FILE_EXTENSION}"
)

In [ ]:
%%writefile $INPUT_FILE
{"data": { "video_uri": VIDEO_PATH_1}}
{"data": { "video_uri": VIDEO_PATH_2}}

In [ ]:
!gsutil cp "instances.jsonl" f"gs://{GCS_BUCKET}"

In [ ]:
gcs_input_uri = f"gs://{GCS_BUCKET}/instances.jsonl"
dest_uri = f"gs://{GCS_BUCKET}/{OUTPUT_GCS_FOLDER}"
print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 创建批量预测作业ID

In [ ]:
JOB_PREFIX = "<job name prefix>"  # @param {type:"string"}
job_name = get_job_name_with_datetime(JOB_PREFIX)
print(job_name)

### 进行批量预测

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name=job_name,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=dest_uri,
    sync=False,
    machine_type=PREDICTION_MACHINE_TYPE,
    service_account=SERVICE_ACCOUNT,
)

print(batch_predict_job)

In [ ]:
batch_predict_job.wait()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

In [ ]:
# Undeploy model and delete endpoint
endpoint.delete(force=True)

# Delete the model resource
model.delete()

delete_bucket = False

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI